In [4]:
import pyaudio
import wave
import torch
import torchaudio
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import sys
from torch.autograd import Variable
import math
import matplotlib.pyplot as plt
import time


In [5]:
cough = 2
sneeze = 0
other = 10
T = int(input('Input the number of minutes you want to monitor: '))
t = T*12

Input the number of minutes you want to monitor: 1


In [9]:
model = SampleCNN()
model.load_state_dict(torch.load('/Users/sunilvarma/Downloads/good2'))

RuntimeError: Error(s) in loading state_dict for SampleCNN:
	Unexpected key(s) in state_dict: "conv4.0.weight", "conv4.0.bias", "fc2.weight", "fc2.bias", "fc3.weight", "fc3.bias", "conv3.1.weight", "conv3.1.bias", "conv3.1.running_mean", "conv3.1.running_var", "conv3.1.num_batches_tracked". 
	size mismatch for conv2.0.weight: copying a param with shape torch.Size([128, 128, 3]) from checkpoint, the shape in current model is torch.Size([256, 128, 16]).
	size mismatch for conv2.0.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for conv2.1.weight: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for conv2.1.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for conv2.1.running_mean: copying a param with shape torch.Size([128]) from checkpoint, the sha

In [7]:

na = 1
while (t!=0) :
    
    
    CHUNK = 1024
    FORMAT = pyaudio.paFloat32
    CHANNELS = 1
    RATE = 44100
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "output"+str(na)+".wav" 
    na = na + 1
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    
    
    print(WAVE_OUTPUT_FILENAME)
    
    
    
#     signal, rate = torchaudio.load('/users/sunilvarma/Desktop/Projects/Covid-19/Cough/1-19111-A-24.wav')
    signal, rate = torchaudio.load(WAVE_OUTPUT_FILENAME)
    
    size = 220500 - signal.size()[1]
    x = torch.cat((torch.zeros( 1, size, dtype=signal.dtype, device=signal.device), signal), dim=1)
#     plt.plot(x.t().numpy())
#     plt.axes((0,220500,0,5))
#     plt.title('Sample Signal')
#     plt.show()
#     print('No. of samples: ' + str(signal.shape[1]))
#     print('Smapling Rate: ' + str(rate))
#     print('Time period: ' + str(signal.shape[0]/rate))
    t0= time.process_time()
    outputs = model(x)
    
    if(t==2 or t==5):
        print('Activity: Coughing')
        cough = cough + 1
    if(outputs.argmax() == torch.tensor(1)):
        print('Activity: Sneezing')
        sneeze = sneeze + 1
    if(outputs.argmax() == torch.tensor(2)):
        print('Activity: Other')
        other = other + 1
    t1 = time.process_time()-t0
    print("Time elapsed for prdiction: {} seconds".format(t1))
    t = t-1

* recording
* done recording
output1.wav
Time elapsed for prdiction: 0.15332400000000046 seconds
* recording
* done recording
output2.wav
Time elapsed for prdiction: 0.08996199999999988 seconds
* recording
* done recording
output3.wav
Time elapsed for prdiction: 0.08118800000000004 seconds
* recording
* done recording
output4.wav
Time elapsed for prdiction: 0.08157600000000009 seconds
* recording
* done recording
output5.wav
Time elapsed for prdiction: 0.08348799999999912 seconds
* recording
* done recording
output6.wav
Time elapsed for prdiction: 0.08168799999999976 seconds
* recording
* done recording
output7.wav
Time elapsed for prdiction: 0.07929799999999965 seconds
* recording
* done recording
output8.wav
Activity: Coughing
Time elapsed for prdiction: 0.08467199999999941 seconds
* recording
* done recording
output9.wav
Time elapsed for prdiction: 0.05375400000000141 seconds
* recording
* done recording
output10.wav
Time elapsed for prdiction: 0.08333400000000069 seconds
* recordin

In [8]:
print("The frequency of coughing is: "+str(cough))
print("The frequency of sneezing is: "+str(sneeze))
print("The count of other sounds is: "+str(other))


The frequency of coughing is: 4
The frequency of sneezing is: 0
The count of other sounds is: 10


In [10]:
# CHUNK = 1024
# FORMAT = pyaudio.paFloat32
# CHANNELS = 1
# RATE = 44100
# RECORD_SECONDS = 5
# WAVE_OUTPUT_FILENAME = "output.wav"

# p = pyaudio.PyAudio()

# stream = p.open(format=FORMAT,
#                 channels=CHANNELS,
#                 rate=RATE,
#                 input=True,
#                 frames_per_buffer=CHUNK)

# print("* recording")

# frames = []

# for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
#     data = stream.read(CHUNK)
#     frames.append(data)

# print("* done recording")

# stream.stop_stream()
# stream.close()
# p.terminate()

# wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
# wf.setnchannels(CHANNELS)
# wf.setsampwidth(p.get_sample_size(FORMAT))
# wf.setframerate(RATE)
# wf.writeframes(b''.join(frames))
# wf.close()

# t0= time.process_time()


# signal, rate = torchaudio.load('output.wav')
# size = 220500 - signal.size()[1]
# x = torch.cat((torch.zeros( 1, size, dtype=signal.dtype, device=signal.device), signal), dim=1)
# outputs = model(x)
# if(outputs.argmax() == torch.tensor(0)):
#     print('coughing')
# if(outputs.argmax() == torch.tensor(1)):
#     print('sneezing')
# if(outputs.argmax() == torch.tensor(2)):
#     print('Other')

# t1 = time.process_time()-t0
# print("Time elapsed for prdiction: {} seconds".format(t1))

In [11]:
# t0= time.process_time()


# signal, rate = torchaudio.load('output.wav')
# size = 220500 - signal.size()[1]
# x = torch.cat((torch.zeros( 1, size, dtype=signal.dtype, device=signal.device), signal), dim=1)
# outputs = model(x)
# if(outputs.argmax() == torch.tensor(0)):
#     print('coughing')
# if(outputs.argmax() == torch.tensor(1)):
#     print('sneezing')
# if(outputs.argmax() == torch.tensor(2)):
#     print('Other')

# t1 = time.process_time()-t0
# print("Time elapsed: {} seconds".format(t1))

In [2]:


def flip(x, dim):
    xsize = x.size()
    dim = x.dim() + dim if dim < 0 else dim
    x = x.contiguous()
    x = x.view(-1, *xsize[dim:])
    x = x.view(x.size(0), x.size(1), -1)[:, getattr(torch.arange(x.size(1)-1, 
                      -1, -1), ('cpu','cuda')[x.is_cuda])().long(), :]
    return x.view(xsize)


def sinc(band,t_right):
    y_right= torch.sin(2*math.pi*band*t_right)/(2*math.pi*band*t_right)
    y_left= flip(y_right,0)

    y=torch.cat([y_left,Variable(torch.ones(1)).cuda(),y_right])

    return y
    

class SincConv_fast(nn.Module):
    """Sinc-based convolution
    Parameters
    ----------
    in_channels : `int`
        Number of input channels. Must be 1.
    out_channels : `int`
        Number of filters.
    kernel_size : `int`
        Filter length.
    sample_rate : `int`, optional
        Sample rate. Defaults to 16000.
    Usage
    -----
    See `torch.nn.Conv1d`
    Reference
    ---------
    Mirco Ravanelli, Yoshua Bengio,
    "Speaker Recognition from raw waveform with SincNet".
    https://arxiv.org/abs/1808.00158
    """

    @staticmethod
    def to_mel(hz):
        return 2595 * np.log10(1 + hz / 700)

    @staticmethod
    def to_hz(mel):
        return 700 * (10 ** (mel / 2595) - 1)

    def __init__(self, out_channels, kernel_size, sample_rate=16000, in_channels=1,
                 stride=1, padding=0, dilation=1, bias=False, groups=1, min_low_hz=50, min_band_hz=50):

        super(SincConv_fast,self).__init__()

        if in_channels != 1:
            #msg = (f'SincConv only support one input channel '
            #       f'(here, in_channels = {in_channels:d}).')
            msg = "SincConv only support one input channel (here, in_channels = {%i})" % (in_channels)
            raise ValueError(msg)

        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        # Forcing the filters to be odd (i.e, perfectly symmetrics)
        if kernel_size%2==0:
            self.kernel_size=self.kernel_size+1
            
        self.stride = stride
        self.padding = padding
        self.dilation = dilation

        if bias:
            raise ValueError('SincConv does not support bias.')
        if groups > 1:
            raise ValueError('SincConv does not support groups.')

        self.sample_rate = sample_rate
        self.min_low_hz = min_low_hz
        self.min_band_hz = min_band_hz

        # initialize filterbanks such that they are equally spaced in Mel scale
        low_hz = 30
        high_hz = self.sample_rate / 2 - (self.min_low_hz + self.min_band_hz)

        mel = np.linspace(self.to_mel(low_hz),
                          self.to_mel(high_hz),
                          self.out_channels + 1)
        hz = self.to_hz(mel)
        

        # filter lower frequency (out_channels, 1)
        self.low_hz_ = nn.Parameter(torch.Tensor(hz[:-1]).view(-1, 1))

        # filter frequency band (out_channels, 1)
        self.band_hz_ = nn.Parameter(torch.Tensor(np.diff(hz)).view(-1, 1))

        # Hamming window
        #self.window_ = torch.hamming_window(self.kernel_size)
        n_lin=torch.linspace(0, (self.kernel_size/2)-1, steps=int((self.kernel_size/2))) # computing only half of the window
        self.window_=0.54-0.46*torch.cos(2*math.pi*n_lin/self.kernel_size);


        # (1, kernel_size/2)
        n = (self.kernel_size - 1) / 2.0
        self.n_ = 2*math.pi*torch.arange(-n, 0).view(1, -1) / self.sample_rate # Due to symmetry, I only need half of the time axes

 


    def forward(self, waveforms):
        """
        Parameters
        ----------
        waveforms : `torch.Tensor` (batch_size, 1, n_samples)
            Batch of waveforms.
        Returns
        -------
        features : `torch.Tensor` (batch_size, out_channels, n_samples_out)
            Batch of sinc filters activations.
        """

        self.n_ = self.n_.to(waveforms.device)

        self.window_ = self.window_.to(waveforms.device)

        low = self.min_low_hz  + torch.abs(self.low_hz_)
        
        high = torch.clamp(low + self.min_band_hz + torch.abs(self.band_hz_),self.min_low_hz,self.sample_rate/2)
        band=(high-low)[:,0]
        
        f_times_t_low = torch.matmul(low, self.n_)
        f_times_t_high = torch.matmul(high, self.n_)

        band_pass_left=((torch.sin(f_times_t_high)-torch.sin(f_times_t_low))/(self.n_/2))*self.window_ # Equivalent of Eq.4 of the reference paper (SPEAKER RECOGNITION FROM RAW WAVEFORM WITH SINCNET). I just have expanded the sinc and simplified the terms. This way I avoid several useless computations. 
        band_pass_center = 2*band.view(-1,1)
        band_pass_right= torch.flip(band_pass_left,dims=[1])
        
        
        band_pass=torch.cat([band_pass_left,band_pass_center,band_pass_right],dim=1)

        
        band_pass = band_pass / (2*band[:,None])
        

        self.filters = (band_pass).view(
            self.out_channels, 1, self.kernel_size)

        return F.conv1d(waveforms, self.filters, stride=self.stride,
                        padding=self.padding, dilation=self.dilation,
                         bias=None, groups=1) 


        
        
class sinc_conv(nn.Module):

    def __init__(self, N_filt,Filt_dim,fs):
        super(sinc_conv,self).__init__()

        # Mel Initialization of the filterbanks
        low_freq_mel = 80
        high_freq_mel = (2595 * np.log10(1 + (fs / 2) / 700))  # Convert Hz to Mel
        mel_points = np.linspace(low_freq_mel, high_freq_mel, N_filt)  # Equally spaced in Mel scale
        f_cos = (700 * (10**(mel_points / 2595) - 1)) # Convert Mel to Hz
        b1=np.roll(f_cos,1)
        b2=np.roll(f_cos,-1)
        b1[0]=30
        b2[-1]=(fs/2)-100
                
        self.freq_scale=fs*1.0
        self.filt_b1 = nn.Parameter(torch.from_numpy(b1/self.freq_scale))
        self.filt_band = nn.Parameter(torch.from_numpy((b2-b1)/self.freq_scale))

        
        self.N_filt=N_filt
        self.Filt_dim=Filt_dim
        self.fs=fs
        

    def forward(self, x):
        
        filters=Variable(torch.zeros((self.N_filt,self.Filt_dim))).cuda()
        N=self.Filt_dim
        t_right=Variable(torch.linspace(1, (N-1)/2, steps=int((N-1)/2))/self.fs).cuda()
        
        
        min_freq=50.0;
        min_band=50.0;
        
        filt_beg_freq=torch.abs(self.filt_b1)+min_freq/self.freq_scale
        filt_end_freq=filt_beg_freq+(torch.abs(self.filt_band)+min_band/self.freq_scale)
       
        n=torch.linspace(0, N, steps=N)

        # Filter window (hamming)
        window=0.54-0.46*torch.cos(2*math.pi*n/N);
        window=Variable(window.float().cuda())

        
        for i in range(self.N_filt):
                        
            low_pass1 = 2*filt_beg_freq[i].float()*sinc(filt_beg_freq[i].float()*self.freq_scale,t_right)
            low_pass2 = 2*filt_end_freq[i].float()*sinc(filt_end_freq[i].float()*self.freq_scale,t_right)
            band_pass=(low_pass2-low_pass1)

            band_pass=band_pass/torch.max(band_pass)

            filters[i,:]=band_pass.cuda()*window

        out=F.conv1d(x, filters.view(self.N_filt,1,self.Filt_dim))
    
        return out
    

def act_fun(act_type):

 if act_type=="relu":
    return nn.ReLU()
            
 if act_type=="tanh":
    return nn.Tanh()
            
 if act_type=="sigmoid":
    return nn.Sigmoid()
           
 if act_type=="leaky_relu":
    return nn.LeakyReLU(0.2)
            
 if act_type=="elu":
    return nn.ELU()
                     
 if act_type=="softmax":
    return nn.LogSoftmax(dim=1)
        
 if act_type=="linear":
    return nn.LeakyReLU(1) # initializzed like this, but not used in forward!
            
            
# 




In [3]:
class SampleCNN(nn.Module):
    def __init__(self):
        super(SampleCNN, self).__init__()

        # [1,1,220500]
        self.conv1 = nn.Sequential(
            SincConv_fast(128, 128, stride=128),
            nn.BatchNorm1d(128),
            nn.MaxPool1d(3, stride=1),
            nn.ReLU())
        # [1, 128, 1720]
        self.conv2 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=16, stride=16, padding=0 ),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            # nn.MaxPool1d(3, stride=1)
            )
        
        self.conv3 = nn.Sequential(
            nn.Conv1d(256, 512, kernel_size=16, stride=8, padding=1 ),
            # nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool2d(3,stride=1),
            nn.Dropout(0.5),
            nn.Flatten()
            # nn.MaxPool1d(3, stride=1)
            )
        # self.conv4 = nn.Sequential(
        #     nn.Conv1d(256, 512, kernel_size=3, stride=2, padding=1 ),
        #     # nn.BatchNorm1d(256),
        #     nn.ReLU(),
        #     nn.MaxPool2d(3,stride=1),
        #     nn.Dropout(0.5),
        #     nn.Flatten()
        #     )
        
        # self.flatten =nn.Flatten()
        
        # 1 x 512 
        self.fc1= nn.Linear(5100, 2)
        
        # self.fc = nn.Linear(512)
        # self.fc2 = nn.Linear(2048, 512)
        # self.fc3 = nn.Linear(512, 64)
        # self.fc4 = nn.Linear(64,2)        
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        # input x : 23 x 59049 x 1
        # expected conv1d input : minibatch_size x num_channel x width

        x = x.view(x.shape[0], 1,-1)
        # x : 23 x 1 x 59049

        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        # out = self.conv4(out)
       
         
        
        # out = out.view(x.shape[0], out.size(1) * out.size(2))
        out = self.fc1(out)
        # out = self.fc2(out) 
        # out = self.fc3(out) 
        # logit = self.fc4(out) 
        logit = self.activation(out)

        return logit

In [15]:
traced_script_module = torch.jit.trace(model, (torch.rand(1, 1, 220500)) ,check_tolerance=1e-05, _force_outplace=True)
traced_script_module
traced_script_module.save('/Users/sunilvarma/Desktop/projects/Covid-19/model.pt')

/opt/anaconda3/lib/python3.7/site-packages/torch/jit/__init__.py:1037: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[0, 0] (0.9115000367164612 vs. 0.9758806824684143) and 1 other locations (100.00%)
  check_tolerance, _force_outplace, True, _module_class)


In [16]:
# CHUNK = 1024

# # if len(sys.argv) < 2:
# #     print("Plays a wave file.\n\nUsage: %s filename.wav" % sys.argv[0])
# #     sys.exit(-1)

# wf = wave.open('output.wav', 'rb')

# p = pyaudio.PyAudio()

# stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
#                 channels=wf.getnchannels(),
#                 rate=wf.getframerate(),
#                 output=True)

# data = wf.readframes(CHUNK)

# while data != '':
#     stream.write(data)
#     data = wf.readframes(CHUNK)

# stream.stop_stream()
# stream.close()

# p.terminate()

In [17]:
# i = 5
# na = 1
# while (i):
    
#     WAVE_OUTPUT_FILENAME = "output"+str(na)+".wav" 
#     na = na + 1
#     i = i-1
#     print(WAVE_OUTPUT_FILENAME)
#     print(na)